In [1]:
import os
import sys
import json
import torch
from tqdm import tqdm
from typing import List
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
from train import train

In [2]:
# !python3 -m src.train --config-file configs/saicuna_13b.json --train-file train.jsonl --val-file val.jsonl  --output-dir models/directum_13b

### Тренировка модели на диалогах с персонажем.

Происходит дообучение ааптера из предыдущего этапа.

In [2]:
output_path = './models/recepties_model'
os.makedirs(output_path, exist_ok=True)

In [ ]:
train_config = {
     'config_file':'config/train_recepties_config.json',
     'train_file':'datasets/train_receptes.jsonl',
     'val_file': 'datasets/test_receptes.jsonl',
     'model_path': "TheBloke/Llama-2-7B-fp16",
     'adapter_path':"./models/adapter", 
     'report_to':'none',
     'output_dir': output_path
}

In [ ]:
train(**train_config)

### Тренировка модели на диалогах с персонажем.

Происходит дообучение ааптера из предыдущего этапа.

In [ ]:
output_path = './models/granny_model'
os.makedirs(output_path, exist_ok=True)

In [ ]:
train_config['config_file'] = "config/train_granny_config.json"
train_config['adapter_path'] = "./models/recepties_model"
train_config['output_dir'] = output_path

In [ ]:
train(**train_config)

### Перенос обученной модели в работчую директорию.

In [ ]:
# train(config_file='train_config.json',
#       train_file='datasets/train_receptes.jsonl',
#       val_file='datasets/test_receptes.jsonl',
#       model_path="./models/llama2",
#       adapter_path="./models/recepties_model", 
#       report_to='none',
#       # checkpoint=adapter_check_point,
#       output_dir=adapter_path)

In [ ]:
# Настраиваем конфиги для адаптера.

In [ ]:
# Надо сделать так чтобы разморозились веса
# только у адарптера и обучался именно он, а не модель.